## ПРОГНОЗИРОВАНИЕ ВРЕМЕННЫХ РЯДОВ С ИЕРАРХИЧЕСКОЙ СТРУКТУРОЙ

### Оглавление

- Введение
- Обзор литературы
- Теоретичсекая часть: описание моделей
    - AR(3)
    - AR(3) + structural break 
    - NOPE AR(3)LI with neighbors (different metrics)
    - Взвешенные прогнозы
    - HB (Иерархический Байес с выделением разных групп  по разным метрикам кластеризации)
- Практическая часть 
    - Данные 
    - CV
        - [How To Backtest Machine Learning Models for Time Series Forecasting](https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/)
        - [Evaluating forecast accuracy](https://otexts.com/fpp2/accuracy.html)
        - Чтобы были сопоставимы, делаем кроссвалидацию с увеличивающимся окном 
    - DM test
        - We cannot compare forecasts for VECTORS of differing (in this case increasing) points in time, we must use a rolling window approach with a fixed h to compare these two models' h-point forecasts along time
        - Also, note that Diebold himself suggests not using Diebold-Mariano test for comparing models rather than forecasts (Diebold "Comparing predictive accuracy, twenty years later: A personal perspective on the use and abuse of Diebold–Mariano tests", 2015).
        - [use and abuse](https://www.sas.upenn.edu/~fdiebold/papers/paper113/Diebold_DM%20Test.pdf)

    - Сравнение моделей




### Введение 

В ананлизе данных часто встречаются данные со сложной многоуровневой структурой. 

Одним из способов повышения точности прогонозов является агрегирование данных. Один из вариантов - агрегирование временных рядов до составления прогноза, другой - агрегирование самих прогнозов. 


Цель:
- Использовать информацию на одном уровне агрегирования для улучшения прогнозов на другом уровне агрегации. 

Задачи: 
- Сбор данных с трехуровневой иерархической структурой
- Сравнение прогнозной силы моделей для агрегированного временного ряда, взвешенных прогнозов дизагрегрованных рядов и иерархической байесовской модели.
- Использование ближайшего по различным метрикам временного ряда в качестве регрессора  
- Добавление в модель дамми структурного сдвига
- Кластеризация временных рядов третьего уровня для получения комбинированных рядов второго уровня (суммирование всех рядов, попавших в один кластер), сравнение прогнозов по  "оригинальным" и "комбинированным" рядам второго уровня


Анализ данных с иерархической структурой требуется в микроэкономике (анализ  спроса на различные виды товаров в разных городах), макроэкономике (ВВП по регонам по разным отраслям), страховании (анализ рисков попасть аварию, в зависимости от привычек и местонахождения человека),  социологии (смертность по регионам и причинам смерти) и т.д.  Помимо этого существует межвременная аггрегация временных рядов, часто применяющаяся при прогнозировании.  



Для анализа были выбраны ряды с определенной структурой, а именно: 
структура трехуровневая и иерархическая, причем просуммировав ряды третьего уровня мы можем получить ряды, как ряды второго уровня, так и сам аггрегированый ряд. В ходе исследования мы определим,  позволяет ли такое детальное дробление временного ряда на более мелкие добиться существенного улучшения прогнозов или можно, например, ограничиться двумя уровнями.
 
 
Добавить из описания данных актуальность
- во первых близкая к 1 корреляция
- точнее считать вклад каждого показателя в агрегат => анализ структуры изменения показтеля
- в среднем значит прогнозы стали лучше для каждого из рядов

http://www.machinelearning.ru/wiki/images/f/fd/2018_June_RZD_presentation.pdf

https://publications.hse.ru/mirror/pubs/share/direct/248447556

- Цели и задачи, определенные в рамках тем в интересах РЖД 
Решается проблема повышения эффективности транспортировки грузов. Для решения задачи выполняется прогноз потребностей узаказчиков РЖД в узлах погрузки/разгрузки с учетомвременных интервалов доставки.

Требуется построить иерархический прогноз объемов спросав заданном периоде:
на месяц посуточно, на месяц подекадно, на квартал помесячно, на год помесячно, на период больше года

С расположением: по группам грузов, по парам станций/регионов, по комбинированному разложению, учитывающему перечисленные варианты

### Обзор литературы




Логично, что каждый временной ряд по-разному реагирует на дополнительные регрессоры, т.е. для каждого ряда мы получим свою оценку коэффициента перед соответвующей объясняющей переменной, в отличие от случая, когда мы получаем оценку при аналогичных регрессорах для аггрегированного ряда, фактически получая одну и ту же оценку для каждого из рядов нижних уровней вне зависимости от того, насколько по-разному они себя ведут со временем.


С другой стороны информация полученная из аггрегированных рядов может иметь существенное влияние при прогнозировании рядов нижнего уровня, хотя ее использование может сопровождаться некоторыми сложностями. 


Для наиболее распространенных моделей прогнозированию существуют альтернативные подходы к анализу временных рядов с иерархической структурой, например, модель  векторной авторегрессии (VAR), в которой  временные ряды имеют общие параметры или модель байесовской векторной авторегрессии (BVAR), где коэффициенты  при различных регрессорах могут иметь общее априорное распределение.  В том числе применяются многомерные модели пространства состояний, векторное  экспоненциальное сглаживание, а также  байесовские подходы, например, их применение к пулу аналогичных временных рядов с помощью ... [Duncan et al. (1993, 2001)].
В таких моделях обычная оценка параметрами объединяется с оцененкой по сгруппированной модели. 

Эмпирические результаты показали, что с помощью перечисленных выше методов точность прогноза может быть улучшена, поскольку они используют ковариационную зависимость  между временными рядами. Однако использование их связано с выполнением большого числа предпосылок или введения соответствующих ограничений на модель.

Эти методы по крайней мере теоретически могут легко обогнать по качеству прогнозов такие простые подходы, как bottom-up (BU), top-down (TD).
Но помимо BU и TD подходов к получению прогнозов аггрегированных рядов, существуют более сложные методы получения  оптимальных комбинаций прогонозов, например, ...
Однако во многих теоретических и эмпирических работах было замечено, что зачастую более простые методы комбинирования прогнозов оказываются в разы эффективнее, сложных методов, использующих метррики, учитывающие особенности каждого из рядов. Так, например, в статье ... лучший прогноз давало простое взвешивание прогнозов. 



### Заключение 



- [Many relevant economic aggregates, like GDP and CPI, do not fall in this category and it is unclear whether these methods will work with relatively small samples.](https://mpra.ub.uni-muenchen.de/81585/1/MPRA_paper_81585.pdf)

- можно провести анализ на увеличивающихся  окнах

- Are there any benefits from using rolling forecasts or recursive filters for prediction?


Примеры иерархических временных рядов:

- Дневные
    - вклад в какой нибудь индекс  
    - просмотры, сколько заходит на страницу по разделам/по возрасту
- Проверить ряды в m3comp

### Список литературы

### To Read

#### Top-Reads

- [Forecasting analogous time series](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.519.6011&rep=rep1&type=pdf)

- [Forecasting output growth rates and median output growth rates: a hierarchical Bayesian approach](https://onlinelibrary.wiley.com/doi/pdf/10.1002/for.800)

- [Forecasting Short Time Series with the Bayesian Autoregression and the Soft Computing Prior Information](https://link.springer.com/chapter/10.1007/978-3-319-10765-3_10)

- [The use of prior information
in forecast combination ](https://www.sas.upenn.edu/~fdiebold/papers/paper94/DieboldPauly1990.pdf) 

- [Discovering Playing Patterns:
Time Series Clustering of Free-To-Play Game Data](https://arxiv.org/pdf/1710.02268.pdf)

- Weighted clustering

- [Joint Forecast Combination of
Macroeconomic Aggregates and Their
Components](https://mpra.ub.uni-muenchen.de/76556/1/MPRA_paper_76556.pdf)

#### Ликбезы

- https://arxiv.org/pdf/1511.00860.pdf

- http://quantile.ru/09/09-AT.pdf

- http://quantile.ru/11/11-SA.pdf

- http://quantile.ru/03/03-SA.pdf

### Literature

- Forecasting Economic Aggregates Using Dynamic Component Grouping


### To Read

#### Top-Reads

- [Forecasting analogous time series](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.519.6011&rep=rep1&type=pdf)

- [Forecasting output growth rates and median output growth rates: a hierarchical Bayesian approach](https://onlinelibrary.wiley.com/doi/pdf/10.1002/for.800)

- [Forecasting Short Time Series with the Bayesian Autoregression and the Soft Computing Prior Information](https://link.springer.com/chapter/10.1007/978-3-319-10765-3_10)

- [The use of prior information
in forecast combination ](https://www.sas.upenn.edu/~fdiebold/papers/paper94/DieboldPauly1990.pdf) 

- [Discovering Playing Patterns:
Time Series Clustering of Free-To-Play Game Data](https://arxiv.org/pdf/1710.02268.pdf)

- Weighted clustering

- [Joint Forecast Combination of
Macroeconomic Aggregates and Their
Components](https://mpra.ub.uni-muenchen.de/76556/1/MPRA_paper_76556.pdf)

#### Ликбезы

- https://arxiv.org/pdf/1511.00860.pdf

- http://quantile.ru/09/09-AT.pdf

- http://quantile.ru/11/11-SA.pdf

- http://quantile.ru/03/03-SA.pdf

### Literature

- Forecasting Economic Aggregates Using Dynamic Component Grouping


#### Данные 

- https://ec.europa.eu/eurostat/web/main/home Eurostat: european statistics
- https://ec.europa.eu/eurostat/cache/metadata/en/nama10_esms.htm

- https://www.fedstat.ru ЕМИСС: государственная статистика
- http://www.gks.ru/bgd/regl/B17_16/Main.htm
- http://www.ukrcensus.gov.ua Держастат Украины

- https://fred.stlouisfed.org FRED Economic Data
- [Gross Domestic Product by State Estimation Methodology ](https://www.bea.gov/sites/default/files/methodologies/0417_GDP_by_State_Methodology.pdf)
- [An Overview of BEA's Source Data and Estimating Methods for Quarterly GDP](https://www.bea.gov/system/files/papers/P2006-8.pdf)
- [Concepts and Methods of the U.S. National Income and Product Accounts](https://www.bea.gov/sites/default/files/methodologies/nipa-handbook-all-chapters.pdf)

dm

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.454.4490&rep=rep1&type=pdf

https://www.sas.upenn.edu/~fdiebold/papers/paper113/Diebold_DM%20Test.pdf
